In [ ]:
def makePRHObs(boris, prh):
    # intialize empty structure
    prhObs = pd.DataFrame()
    prhObs['DN'] = pd.Series(prh.get('DN').squeeze())

    # 02.10.2024. HMC. Wasn't calculating the time right, changed code to correct
    def matlab_datenum_to_datetime(matlab_datenum):
        python_datetime = datetime.fromordinal(int(matlab_datenum)) + timedelta(days=matlab_datenum % 1) - timedelta(days=366)
        return python_datetime
    prhObs['DS'] = prhObs['DN'].apply(matlab_datenum_to_datetime)
    #prhObs['DS'] = prhObs.apply(lambda x: datetime.datetime.fromtimestamp((x['DN']- 719529) * 86400).strftime('%Y-%m-%d %H:%M:%S.%f'), axis=1)
    prhObs['DS'] = pd.to_datetime(prhObs['DS'])  
    prhObs['ind'] = prhObs.index
    prhObs['boolObs'] = np.nan # whether an observation occurs at this index
    prhObs['boolSocial'] = np.nan  # is it social
    prhObs['boolPrey']= np.nan  # WAS PREY OBSERVED
    prhObs['boolFeed']= np.nan  # WAS PREY OBSERVED
    #prhObs['boolRest']= np.nan  # WAS RESTING OBSERVED
    prhObs['ObsType']= ''
    prhObs['camon'] = pd.Series(prh.get('camon').squeeze()) # GET BOOL CAMON FROM PRH

    periodsNOCAM = pd.DataFrame()
    nocamSI = []
    nocamEI = []
    #loop through each sample of camon
    for i,v in enumerate(prhObs['camon']):
        # if this is not the first index
        if i != 0:
            lastval = prhObs['camon'].iloc[i-1]
            diff = lastval - v
            if lastval != v:
                if diff < 0:
                    nocamEI.append(i-1)
                elif diff == 1:
                    nocamSI.append(i)
        else:
            # if this is the first iteration, and the camera was not on, add this index (the first) to the no cam start indices
            if v != 1:
                nocamSI.append(i)
            else:
                continue
            lastval = np.nan

        # get the value of the last index
    if len(nocamSI) != len(nocamEI):
        nocamEI.append(len(prhObs)-1)
    periodsNOCAM['sI'] = nocamSI
    periodsNOCAM['eI'] = nocamEI
    periodsNOCAM = periodsNOCAM.merge(prhObs, left_on='sI', right_index=True, how='left', suffixes=('', '_s'))
    periodsNOCAM = periodsNOCAM.merge(prhObs, left_on='eI', right_index=True, how='left', suffixes=('', '_e'))

    # Rename the columns
    periodsNOCAM = periodsNOCAM.rename(columns={'DS': 'sDS', 'DS_e': 'eDS'})
    keepcol = ['sI', 'eI', 'sDS', 'eDS']
    periodsNOCAM = periodsNOCAM[keepcol]

    obslist = boris['ObsInd'].astype(int).tolist()
    #obslist = [int(item) for item in obslist]
    prhObs['boolObs'].iloc[obslist] = True
    #prhObs['boolObs'] = prhObs['boolObs'].replace(np.nan, False)
    prhObs['boolObs'].fillna(False, inplace=True)

    # SOCIAL
    soclist = np.isin(boris, ['Physical Contact','No Contact Social Interaction'])
    soclist = list(np.where(np.any(soclist==True, axis = 1)))
    soclist = boris['ObsInd'].iloc[soclist[0]].astype(int)
    prhObs['boolSocial'].iloc[soclist] = bool(True)
    prhObs['boolSocial'] = prhObs['boolSocial'].replace(np.nan, False)
    prhObs['ObsType'].iloc[soclist] = str('Social')

    #PREY
    preylist = np.isin(boris, ['Ghost Shrimp Burrows', 'Prey'])
    preylist = list(np.where(np.any(preylist==True, axis = 1)))
    preylist = boris['ObsInd'].iloc[preylist[0]].astype(int)
    prhObs['boolPrey'].iloc[preylist] = bool(True)
    prhObs['boolPrey'] = prhObs['boolPrey'].replace(np.nan, False)
    prhObs['ObsType'].iloc[preylist] = str('Prey')

    #FEEDING
    feedlist = np.isin(boris, ['Pit','Pit Comment', 'Begin Stationary Foraging', 'End Stationary Foraging'])
    feedlist = list(np.where(np.any(feedlist==True, axis = 1)))
    feedlist = boris['ObsInd'].iloc[feedlist[0]].astype(int)
    prhObs['boolFeed'].iloc[feedlist] = bool(True)
    prhObs['boolFeed'] = prhObs['boolFeed'].replace(np.nan, False)
    prhObs['ObsType'].iloc[feedlist] = str('Feed')

    #RESTING
    #restlist = np.isin(boris, ['(Bottom)'])
    #restlist = list(np.where(np.any(restlist==True, axis = 1)))
    #restlist = boris['ObsInd'].iloc[restlist[0]].astype(int)
    #prhObs['boolRest'].iloc[feedlist] = bool(True)
    #prhObs['boolRest'] = prhObs['boolRest'].replace(np.nan, False)
    #prhObs['ObsType'].iloc[feedlist] = str('Rest')

    #prhOnlyObs = prhObs[prhObs['ObsType'].notnull()]

    #PREP DATA FOR PLOTTING
    tagon = prh.get('tagon')
    indTagOn = np.argmax(tagon == 1)
    tagonDS = prhObs['DS'][indTagOn]
    indices_of_ones = np.where(tagon == 1)[0]
    if len(indices_of_ones) > 0:
        tagoffInd = indices_of_ones[-1]
    else:
        print("No occurrence of '1' in tagon array.")
    tagoffDS = prhObs['DS'][tagoffInd]

    prhObs['p'] = prh.get('p')
    prhObs['roll'] = prh.get('roll')
    periodsNOCAM.to_excel(outloc+depID+"-periodsNOCAM.xlsx")

    return(prhObs, indTagOn, tagoffInd, tagoffDS, tagonDS, periodsNOCAM)

            



    